In [38]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import geopandas as gpd
import statsmodels.formula.api as smf
from plotnine import *
pd.options.mode.chained_assignment = None
energy = pd.read_csv("https://data.cityofnewyork.us/api/views/5zyy-y8am/rows.csv?date=20231212&accessType=DOWNLOAD")
covered_buildings = pd.read_csv("../data/covered_buildings.csv")

energy['BBL'] = energy['NYC Borough, Block and Lot (BBL)'].str.replace('-', '')
#energy = energy[energy['BBL'].isin(covered_buildings['10 Digit BBL'].astype(str))]

/var/folders/ct/9lwc0fbn26v_v6pz1pfj1dxc0000gq/T/ipykernel_47548/3761795420.py:10: DtypeWarning: Columns (10,78) have mixed types. Specify dtype option on import or set low_memory=False.


In [39]:
energy["Multifamily Housing - Number of Laundry Hookups in All Units"]
energy["Multifamily Housing - Number of Laundry Hookups in Common Area(s)"]

0        Not Available
1        Not Available
2        Not Available
3                    4
4        Not Available
             ...      
30480    Not Available
30481    Not Available
30482    Not Available
30483    Not Available
30484    Not Available
Name: Multifamily Housing - Number of Laundry Hookups in Common Area(s), Length: 30485, dtype: object

In [40]:
housing = energy[energy['Primary Property Type - Portfolio Manager-Calculated'] == "Multifamily Housing"]
housing['total_units'] = pd.to_numeric(housing['Multifamily Housing - Total Number of Residential Living Units'], errors='coerce')
housing['co2_intensity'] = pd.to_numeric(housing['Total (Location-Based) GHG Emissions Intensity (kgCO2e/ft²)'], errors='coerce')
housing['EUI'] = pd.to_numeric(housing['Site EUI (kBtu/ft²)'], errors='coerce')
housing['housing_fa'] = pd.to_numeric(housing['Multifamily Housing - Gross Floor Area (ft²)'], errors='coerce')
housing['total_fa'] = pd.to_numeric(housing['Property GFA - Self-Reported (ft²)'], errors='coerce')
housing["total_co2"] = pd.to_numeric(housing['Total (Location-Based) GHG Emissions (Metric Tons CO2e)'], errors='coerce') 
housing["bedrooms"] = pd.to_numeric(housing["Multifamily Housing - Number of Bedrooms"], errors='coerce')
housing["direct_emissions"] =  pd.to_numeric(housing["Direct GHG Emissions (Metric Tons CO2e)"], errors='coerce')
housing["indirect_emissions"] = pd.to_numeric(housing["Indirect (Location-Based) GHG Emissions (Metric Tons CO2e)"], errors='coerce')
housing["kwh"] = pd.to_numeric(housing["Electricity Use - Grid Purchase (kWh)"], errors='coerce')
housing["gallons_water"] = pd.to_numeric(housing["Water Use (All Water Sources) (kgal)"], errors='coerce') * 1000
housing["unit_laundry_hookups"] = pd.to_numeric(energy["Multifamily Housing - Number of Laundry Hookups in All Units"], errors="coerce")
housing["common_laundry_hookups"] = pd.to_numeric(energy["Multifamily Housing - Number of Laundry Hookups in Common Area(s)"], errors="coerce")


In [41]:
housing[["unit_laundry_hookups", "common_laundry_hookups"]].sample(20)


,unit_laundry_hookups,common_laundry_hookups
9323,0.0,0.0
17390,138.0,10.0
12099,0.0,2.0
19169,0.0,17.0
25863,0.0,0.0
4338,NaN,NaN
13144,NaN,NaN
10469,NaN,NaN
5964,NaN,NaN
3850,0.0,0.0


In [42]:
housing_trim = housing[['BBL', 'Property Name', 'Property ID', 'total_units', 'EUI', 'bedrooms', 
                        'housing_fa', 'total_fa', 'co2_intensity', "total_co2", "kwh", 'gallons_water', 'Latitude', 
                        'Longitude', "Year Built", "Occupancy", "unit_laundry_hookups", "common_laundry_hookups",
                        "direct_emissions", "indirect_emissions"]].rename(columns={'Property Name': 'name', 'Census Tract' : "GEOID", "Year Built": "year_built"}).query("total_units > 0")

housing_trim['total_E'] = housing_trim['EUI'] * housing_trim['total_fa']
housing_trim["kwh_per_sqft"] = housing_trim['kwh'] / housing_trim['total_fa']
housing_trim['avg_apt_size'] = housing_trim['housing_fa'] / housing_trim['total_units']
housing_trim['E_per_apt'] = housing_trim['total_E'] / housing_trim['total_units']
housing_trim['co2_tons_sq'] = housing_trim['co2_intensity'] / 1000
housing_trim['co2_per_apt'] = (housing_trim['co2_tons_sq'] * housing_trim['total_fa']) / housing_trim['total_units']
housing_trim['co2_per_bedroom'] = (housing_trim['co2_tons_sq'] * housing_trim['total_fa']) / housing_trim['bedrooms']
housing_trim['exceeds_2024_limit'] = housing_trim['co2_tons_sq'] > 0.00675
housing_trim['exceeds_2030_limit'] = housing_trim['co2_tons_sq'] > 0.00407
housing_trim['decade_built'] = (np.floor(housing_trim["year_built"]/10)*10).astype(int)
housing_trim['bedrooms_per_apt'] =  housing_trim["bedrooms"] / housing_trim["total_units"]
housing_trim['sqft_per_bedroom'] = housing_trim['housing_fa'] / housing_trim["bedrooms"]
housing_trim["direct_emission_pct"] = housing_trim["direct_emissions"] / housing_trim["total_co2"]
housing_trim["water_per_sqft"] = housing_trim["gallons_water"] / housing_trim['total_fa']
housing_trim["water_per_apt"] = housing_trim["gallons_water"] / housing_trim['total_units']
housing_trim["laundry_per_apt"] = housing_trim["unit_laundry_hookups"] / housing_trim['total_units']

In [43]:
housing_trim["laundry_cat"] = np.select(
    [
        housing_trim["common_laundry_hookups"].isna(),
        housing_trim["laundry_per_apt"] > .5,
        housing_trim["common_laundry_hookups"] > 0,
    ],
    [
        "Unknown",
        "In-unit",
        "In-building"
    ],
    default = "None"

)

In [44]:
housing_trim["efficiency_quintile"] = pd.qcut(housing_trim["co2_tons_sq"], 5, labels=[1,2,3,4,5])
housing_trim["unit_co2_quintile"] = pd.qcut(housing_trim["co2_per_apt"], 5, labels=[1,2,3,4,5])
housing_trim["bedroom_co2_quintile"] = pd.qcut(housing_trim["co2_per_bedroom"], 5, labels=[1,2,3,4,5])
housing_trim["electricty_sqft_quintile"] = pd.qcut(housing_trim["kwh_per_sqft"], 5, labels=[1,2,3,4,5])

/Users/canyonfoot/Documents/python_proj/Building-Emissions/.venv/lib/python3.10/site-packages/numpy/lib/function_base.py:4655: RuntimeWarning: invalid value encountered in subtract


In [45]:
# 1 row has missing value.. 11 have inf
housing_trim = housing_trim.dropna(subset=["sqft_per_bedroom"])

In [46]:
housing_trim = housing_trim[(housing_trim["housing_fa"] / housing_trim["total_fa"]) > .9]

In [47]:
conditions = [
    (housing_trim['exceeds_2024_limit'] == True),
    (housing_trim['exceeds_2024_limit'] == False) & (housing_trim['exceeds_2030_limit'] == True),
    (housing_trim['exceeds_2024_limit'] == False) & (housing_trim['exceeds_2030_limit'] == False)
]

# Define the choices
choices = ['Exceeds 2024 limit', 'Exceeds 2030 limit', 'Below both limits']

# Create the new column
housing_trim['limit_category'] = np.select(conditions, choices)

In [48]:
housing_trim['apt_size_cat'] = pd.cut(
    housing_trim['avg_apt_size'],
    [0, 750, 1000, 1250, 2000, np.inf],
    labels=["< 750", "750 - 1000", "1000 - 1250", "1250 - 2000", "2000 +"]
)

housing_trim['apt_size_cat2'] = pd.cut(
    housing_trim['avg_apt_size'],
    [0, 650, 1000, 1250, np.inf],
    labels=["< 650", "650 - 1000", "1000 - 1250", "1250+"]
)

housing_trim['sqft_bedroom_cat'] = pd.cut(
    housing_trim['sqft_per_bedroom'],
    [0, 450, 600, 750, 1000, np.inf],
    labels=["< 450", "450 - 600", "600 - 750", "750 - 1000", "1000 +"]
)

In [49]:
import censusdis.data as ced
import censusdis.geography as cgeo
from censusdis.states import STATE_NY
YEAR = 2020
DATASET = "acs/acs5"
GROUP = "B03002"
CENSUS_VARS = {
    "B17001_001E" : "total_population",
    "B17001_002E" : "pop_in_poverty",
    "B03002_004E" : "pop_black",
    "B03002_012E" : "pop_hispanic",
    "B19013_001E" : "med_hh_inc"
}

In [50]:
nyc_tracts = ced.download(
    DATASET,
    YEAR,
    CENSUS_VARS,
    state=STATE_NY,
    county="*",
    tract = "*",
    with_geometry = True
).rename(CENSUS_VARS, axis = 1)
nyc_tracts["GEOID"] = nyc_tracts["STATE"] + nyc_tracts["COUNTY"] + nyc_tracts["TRACT"]

In [51]:
cols_to_divide = [col for col in nyc_tracts.columns if col.startswith('pop')]
nyc_tracts[cols_to_divide] = nyc_tracts[cols_to_divide].div(nyc_tracts['total_population'], axis=0)

In [52]:
from shapely.geometry import Point

# Create a new geometry column in your DataFrame
housing_trim['geometry'] = housing_trim.apply(lambda row: Point(row.Longitude, row.Latitude), axis=1)

# Convert your DataFrame to a GeoDataFrame
housing_trim_geo = gpd.GeoDataFrame(housing_trim, geometry='geometry', crs = "EPSG:4269")

In [53]:
housing_trim_geo = housing_trim_geo.overlay(nyc_tracts.to_crs(4269))

In [54]:
housing_trim_geo = housing_trim_geo.query("med_hh_inc > 0")

In [55]:
bins = [0, 30000, 75000, 125000, 175000, np.inf]
labels = ['<30k', '30k-75k', '75k-125k', '125k-175k', '175k+']

# Create a new column with the binned data
housing_trim_geo['income_bin'] = pd.cut(housing_trim_geo['med_hh_inc'], bins=bins, labels=labels)
housing_trim_geo["income_q"] = pd.qcut(housing_trim_geo['med_hh_inc'], q=10, labels = False)

In [56]:
coefs = {'Fuel Oil #2 Use (kBtu)' : 0.00007421,
 'Fuel Oil #4 Use (kBtu)' : 0.00007529,
 'District Steam Use (kBtu)' : 0.00004493,
 'Natural Gas Use (kBtu)' : 0.00005311 ,
 'Electricity Use - Grid Purchase (kWh)' : 0.000288962}

In [57]:
housing["Electricity Use - Grid Purchase (kBtu)"]

1        1642970.2
3        2191128.4
5         703117.6
8        1135718.3
9        1596587.4
           ...    
30462     458361.3
30466     601259.2
30467     618933.4
30477    1169654.1
30478     606776.4
Name: Electricity Use - Grid Purchase (kBtu), Length: 19739, dtype: object

In [58]:
energy_mix = housing[['Property ID', 'Electricity Use - Grid Purchase (kBtu)'] + list(coefs.keys())].apply(pd.to_numeric, errors='coerce').fillna(0)

In [59]:
for col, coef in coefs.items():
    energy_mix[col + "_est"] = energy_mix[col] * coef

In [60]:
# Create 'total' column
cols = [col + "_est" for col in coefs.keys()]

energy_mix['total_emissions'] = energy_mix[cols].sum(axis=1)

# Create proportion columns
for col in cols:
    energy_mix[f'{col}_proportion'] = energy_mix[col] / energy_mix['total_emissions']

In [61]:
energy_mix["fuel_oil_emissions_pct"] = energy_mix["Fuel Oil #2 Use (kBtu)_est_proportion"] + energy_mix["Fuel Oil #4 Use (kBtu)_est_proportion"]
energy_mix["district_steam_emissions_pct"] = energy_mix["District Steam Use (kBtu)_est_proportion"]
energy_mix["natural_gas_emissions_pct"] = energy_mix["Natural Gas Use (kBtu)_est_proportion"]
energy_mix["electricity_emissions_pct"] = energy_mix["Electricity Use - Grid Purchase (kWh)_est_proportion"]

In [62]:
# Create 'total' column
cols = [col for col in energy_mix.columns if col.endswith("(kBtu)")]
cols

['Electricity Use - Grid Purchase (kBtu)',
 'Fuel Oil #2 Use (kBtu)',
 'Fuel Oil #4 Use (kBtu)',
 'District Steam Use (kBtu)',
 'Natural Gas Use (kBtu)']

In [63]:
cols = [col for col in energy_mix.columns if col.endswith("(kBtu)")]

energy_mix['total_energy'] = energy_mix[cols].sum(axis=1)

# Create proportion columns
for col in cols:
    energy_mix[f'{col}energy_proportion'] = energy_mix[col] / energy_mix['total_energy']

In [64]:
energy_mix["fuel_oil_energy_pct"] = energy_mix["Fuel Oil #2 Use (kBtu)energy_proportion"] + energy_mix["Fuel Oil #4 Use (kBtu)energy_proportion"]
energy_mix["district_steam_energy_pct"] = energy_mix["District Steam Use (kBtu)energy_proportion"]
energy_mix["natural_gas_energy_pct"] = energy_mix["Natural Gas Use (kBtu)energy_proportion"]
energy_mix["electricity_energy_pct"] = energy_mix["Electricity Use - Grid Purchase (kBtu)energy_proportion"]

In [65]:

energy_mix['inferred_heating_method'] = np.select(
    [   
        energy_mix['natural_gas_emissions_pct'] > .3,
        energy_mix['fuel_oil_emissions_pct'] > .4, 
        energy_mix['district_steam_emissions_pct'] > .3,
        energy_mix['electricity_emissions_pct'] > .8, 

    ], 
    [
        'Natural Gas',
        'Fuel Oil', 
        'District Steam',
        'Electricity'

    ], 
    default='Unknown'
)

In [66]:
energy_mix

,Property ID,Electricity Use - Grid Purchase (kBtu),Fuel Oil #2 Use (kBtu),Fuel Oil #4 Use (kBtu),District Steam Use (kBtu),Natural Gas Use (kBtu),Electricity Use - Grid Purchase (kWh),Fuel Oil #2 Use (kBtu)_est,Fuel Oil #4 Use (kBtu)_est,District Steam Use (kBtu)_est,...,Electricity Use - Grid Purchase (kBtu)energy_proportion,Fuel Oil #2 Use (kBtu)energy_proportion,Fuel Oil #4 Use (kBtu)energy_proportion,District Steam Use (kBtu)energy_proportion,Natural Gas Use (kBtu)energy_proportion,fuel_oil_energy_pct,district_steam_energy_pct,natural_gas_energy_pct,electricity_energy_pct,inferred_heating_method
1,9793770,1642970.2,0.0,0.0,0.0,1453700.0,481527.0,0.000000,0.0,0.0,...,0.530560,0.000000,0.0,0.0,0.469440,0.000000,0.0,0.469440,0.530560,Natural Gas
3,13511507,2191128.4,0.0,0.0,0.0,5122899.9,642182.9,0.000000,0.0,0.0,...,0.299579,0.000000,0.0,0.0,0.700421,0.000000,0.0,0.700421,0.299579,Natural Gas
5,14377690,703117.6,0.0,0.0,0.0,5199600.0,206072.0,0.000000,0.0,0.0,...,0.119118,0.000000,0.0,0.0,0.880882,0.000000,0.0,0.880882,0.119118,Natural Gas
8,15176247,1135718.3,414000.0,0.0,0.0,8852599.8,332860.0,30.722940,0.0,0.0,...,0.109179,0.039799,0.0,0.0,0.851022,0.039799,0.0,0.851022,0.109179,Natural Gas
9,15176327,1596587.4,588087.1,0.0,0.0,8681600.0,467932.9,43.641944,0.0,0.0,...,0.146931,0.054120,0.0,0.0,0.798949,0.054120,0.0,0.798949,0.146931,Natural Gas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30462,6699459,458361.3,0.0,0.0,0.0,3067877.2,134338.0,0.000000,0.0,0.0,...,0.129986,0.000000,0.0,0.0,0.870014,0.000000,0.0,0.870014,0.129986,Natural Gas
30466,6681872,601259.2,2347656.0,0.0,0.0,164300.0,176219.0,174.219552,0.0,0.0,...,0.193131,0.754094,0.0,0.0,0.052775,0.754094,0.0,0.052775,0.193131,Fuel Oil
30467,6682201,618933.4,2398577.8,0.0,0.0,153400.0,181399.0,177.998459,0.0,0.0,...,0.195191,0.756432,0.0,0.0,0.048377,0.756432,0.0,0.048377,0.195191,Fuel Oil
30477,6875712,1169654.1,0.0,0.0,0.0,6417199.4,342806.0,0.000000,0.0,0.0,...,0.154169,0.000000,0.0,0.0,0.845831,0.000000,0.0,0.845831,0.154169,Natural Gas


In [67]:
energy_mix_join = energy_mix[["Property ID", "total_energy", "total_emissions", "district_steam_emissions_pct", "natural_gas_emissions_pct",
                               "electricity_emissions_pct", "fuel_oil_emissions_pct", "fuel_oil_energy_pct", "district_steam_energy_pct", "natural_gas_energy_pct", "electricity_energy_pct", "inferred_heating_method"]]
housing_trim_geo = housing_trim_geo.merge(energy_mix_join)

In [68]:
# Convert categorical columns to string
for col in housing_trim_geo.select_dtypes(include='category').columns:
    housing_trim_geo[col] = housing_trim_geo[col].astype(str)

housing_trim_geo.columns = housing_trim_geo.columns.str.lower()

In [69]:
pluto = pd.read_csv("https://data.cityofnewyork.us/api/views/64uk-42ks/rows.csv?date=20231212&accessType=DOWNLOAD")
bor_map = {"BK" : "3",
           "BX" : "2",
           "MN" : "1",
           "SI" : "5",
           "QN" : "4"}
pluto["borough"] = pluto["borough"].map(bor_map)

/var/folders/ct/9lwc0fbn26v_v6pz1pfj1dxc0000gq/T/ipykernel_47548/2150334008.py:1: DtypeWarning: Columns (17,18,20,21,22,24,25,52,61,62,79,97) have mixed types. Specify dtype option on import or set low_memory=False.


In [70]:
pluto['block'] = pluto['block'].apply(lambda x: str(x).zfill(5))
pluto['lot'] = pluto['lot'].apply(lambda x: str(x).zfill(4))
pluto["bbl"] = pluto["borough"] + pluto["block"] + pluto["lot"]

In [71]:
housing_trim_geo = housing_trim_geo.merge(pluto[["bbl", "yearalter1", "yearalter2", "assesstot"]])
housing_trim_geo["ever_altered"] = housing_trim_geo["yearalter1"] != 0
housing_trim_geo["altered_twice_or_more"] = housing_trim_geo["yearalter2"] != 0

In [72]:
housing_trim_geo["value_per_apt"] = housing_trim_geo["assesstot"] / housing_trim_geo["total_units"] / .06
housing_trim_geo["value_per_sqft"] = housing_trim_geo["assesstot"] / housing_trim_geo["total_fa"] / .06

In [73]:
housing_trim_geo.to_file("../data/ghg_processed.geojson")

In [74]:
housing_trim_geo.query("total_co2.isna()")

,bbl,name,property id,total_units,eui,bedrooms,housing_fa,total_fa,co2_intensity,total_co2,...,natural_gas_energy_pct,electricity_energy_pct,inferred_heating_method,yearalter1,yearalter2,assesstot,ever_altered,altered_twice_or_more,value_per_apt,value_per_sqft
3,2042520017,A02760 - Parkway Owners Inc,15289404,52.0,NaN,84.00,44329.0,44329.0,NaN,NaN,...,0.0,0.0,Fuel Oil,0.0,0.0,1729350.0,False,False,5.542788e+05,650.195132
400,4013377503,Woodside Manor Condominiums,8460008,28.0,NaN,41.00,20658.0,20658.0,NaN,NaN,...,NaN,NaN,Unknown,0.0,0.0,1501269.0,False,False,8.936125e+05,1211.208733
401,4013377502,Woodside Terrace Condominiums,8460009,24.0,NaN,46.00,19362.0,19362.0,NaN,NaN,...,1.0,0.0,Natural Gas,0.0,0.0,1515089.0,False,False,1.052145e+06,1304.177427
555,4012220009,AAR: 65-15 38 Avenue,2801835,145.0,NaN,145.00,139100.0,139100.0,NaN,NaN,...,1.0,0.0,Natural Gas,0.0,0.0,3060000.0,False,False,3.517241e+05,366.642703
653,1021360235,FORT WASHINGTON AVENUE REHAB,2831038,226.0,NaN,249.00,293550.0,293550.0,NaN,NaN,...,0.0,1.0,Electricity,0.0,0.0,10984050.0,False,False,8.100332e+05,623.633112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14553,3014330060,2149-53 Pacific St HDFC,16122881,24.0,NaN,47.00,25180.0,25180.0,NaN,NaN,...,1.0,0.0,Natural Gas,1976.0,0.0,733050.0,True,False,5.090625e+05,485.206513
14754,1016780001,325 E 106th St,11346673,117.0,NaN,176.00,124000.0,124000.0,NaN,NaN,...,0.0,1.0,Electricity,0.0,0.0,31421700.0,False,False,4.476026e+06,4223.346774
16134,3012860051,K & J Management CO,6299853,27.2,NaN,31.74,22669.0,22669.0,NaN,NaN,...,1.0,0.0,Natural Gas,0.0,0.0,545850.0,False,False,3.344669e+05,401.318982
16732,3056447501,Sunset Park Manor Condominium,8460007,8.0,NaN,21.83,15591.0,15591.0,NaN,NaN,...,1.0,0.0,Natural Gas,0.0,0.0,440100.0,False,False,9.168750e+05,470.463729
